In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.utils import image_dataset_from_directory
from tensorflow import clip_by_value
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import random as tf_random
import keras_cv
from keras import layers, models
import os
import time
import numpy as np
from matplotlib import pyplot as plt
from keras.regularizers import l1_l2
input_shape = (32, 32, 3)
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, Rescaling

# load data

In [ ]:
train_ds = image_dataset_from_directory(
    '../data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='training',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    label_mode = 'categorical')

class_names = train_ds.class_names

val_ds = image_dataset_from_directory(
    '../data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='validation',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    label_mode = 'categorical')

In [ ]:
# def load_and_preprocess_image(image_path, label):
#     # Load image
#     image = tf.io.read_file(image_path)
#     # Decode PNG image to tensor
#     image = tf.image.decode_png(image, channels=3)  # Adjust channels according to your images
#     # Normalize pixel values to range [0, 1]
#     image = tf.image.convert_image_dtype(image, tf.float32)
#     return image, label


# def preprocess_data():

#     class_names = os.listdir(data_dir)

#     image_paths = []
#     labels = []
#     for class_name in class_names:
#         class_dir = os.path.join(data_dir, class_name)
#         for image_name in os.listdir(class_dir):
#             image_path = os.path.join(class_dir, image_name)
#             image_paths.append(image_path)
#             labels.append(class_names.index(class_name))

#     # Create TensorFlow Dataset from the loaded data
#     dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
#     dataset = dataset.map(load_and_preprocess_image)

#     dataset = dataset.shuffle(buffer_size=10000)

# # Split the dataset into train, validation, and test sets
#     train_size = int(0.8 * len(dataset))
#     test_size = int(0.2 * len(dataset))

#     train_dataset = dataset.take(train_size)
#     test_dataset = dataset.skip(train_size).take(test_size)
#     train_dataset = train_dataset.shuffle(buffer_size=len(image_paths)).batch(32)
#     test_dataset = test_dataset.shuffle(buffer_size=len(image_paths)).batch(32)
#     return train_dataset, test_dataset

# pre-made dataset from cifar dataset

In [ ]:
train_dataset, test_dataset = preprocess_data()

# CNN definition

In [ ]:
def cnn(input_shape, l1=0.01, l2=0.01, dropout_rate=0.2):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=l1_l2(l1, l2))) # output shape is 30x30x32
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2))) # output shape is 15x15x32
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l1_l2(l1, l2))) # output shape is 13x13x64
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2))) # output shape is 6x6x64
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))

    # model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l1_l2(l1, l2)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.MaxPooling2D((2,2)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Dropout(dropout_rate))
    # Dense layers
    model.add(layers.Flatten()) # 1024
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=l1_l2(l1, l2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(10, activation='softmax', kernel_regularizer=l1_l2(l1, l2)))

    return model


In [ ]:
model = cnn(input_shape)
model.summary()

### searching for optimal dropout

In [ ]:
histories = {}
for dropout_rate in np.linspace(0, 1/2, 10):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)    
    model = cnn(input_shape, True, dropout_rate=dropout_rate)
    model.compile(optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'])
    print(f'dropout = {dropout_rate}')
    history = model.fit(train_ds, validation_data = val_ds, epochs = 5)    
    histories[dropout_rate] = {'history': history.history}

In [ ]:
for dropout in histories:
    plt.scatter(dropout, histories[dropout]['history']['val_accuracy'][-1])

In [ ]:
l1_range = np.linspace(0, 1, 10)  # Example: 10 values between 0 and 1
l2_range = np.linspace(0, 1, 10)  # Example: 10 values between 0 and 1

# Generate candidate combinations using grid search
search_space = [(l1, l2) for l1 in l1_range for l2 in l2_range]
for l1, l2 in search_space:
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)    
    model = cnn(input_shape, True, l1, l2)
    model.compile(optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
    print(f'l1 = {l1}, l2 = {l2}')
    history = model.fit(train_dataset, epochs = 5)    
    test_loss, test_acc = model.evaluate(test_dataset)
    histories[(l1, l2)] = {'history': history.history,
                        'test_loss': test_loss,
                        'test_acc': test_acc}

In [ ]:
for (l1, l2) in histories:
    plt.scatter(l1, l2, s = histories[(l1, l2)]['test_acc']*400, c = histories[(l1, l2)]['test_acc'])

In [ ]:
def cnn(dropout_rate = 0.2, regul_alg=tf.keras.regularizers.L1, regul_par=0, optimizer = 'Adagrad'):
    model = models.Sequential()
    model.add(layers.Input((32,32,3)))
    model.add(layers.Rescaling(1./255))

    model.add(layers.Conv2D(64,(4,4),activation='relu', kernel_regularizer=regul_alg(regul_par)))
    # if batch_normalization:
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    
    model.add(layers.Conv2D(128,(4,4),activation='relu', kernel_regularizer=regul_alg(regul_par)))
    # if batch_normalization:
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(1024,activation='relu', kernel_regularizer=regul_alg(regul_par)))
    # if batch_normalization:
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1024,activation='relu', kernel_regularizer=regul_alg(regul_par)))
    # if batch_normalization:
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    
    return model

In [ ]:
#  base model, no dropout, no batch normalization after ReLu layers

In [ ]:
model = cnn(0)
model.summary()

# base model without any augmentations

In [ ]:
# histories = {}
for optimizer in ['SGD', 'Adagrad', 'RMSprop']:
    for dropout in [0.1, 0.2, 0.3]:
        for regularization_algo, regularization_algo_name in zip([tf.keras.regularizers.L1, tf.keras.regularizers.L2], ['l1', 'l2']):
            for l in [0.001, 0.01, 0.1]:
                if (optimizer, dropout, regularization_algo, l) in histories.keys():
                    continue
                print(f"optimizer = {optimizer}, dropout = {dropout}, 'regularization_algo = {regularization_algo}, l = {l}")
                model = cnn(dropout, regularization_algo, l, optimizer)
                history = model.fit(train_ds, validation_data = val_ds, epochs = 3)
                histories[(optimizer, dropout, regularization_algo, l)] = history

In [ ]:
data = {'optimizer' : [], 'regularizer' : [], 'lambda' : [], 'dropout' : [], 'best_acc' : []}
for key, val in histories.items():
    data['optimizer'].append(key[0])
    data['regularizer'].append(key[1])
    data['lambda'].append(key[2])
    data['dropout'].append(key[3])
    data['best_acc'].append(val['accuracy'][-1] if type(val) == dict else val.history['accuracy'][-1])
data

In [ ]:
pd.DataFrame(data).sort_values('best_acc', ascending = False)